<a href="https://colab.research.google.com/github/egerling/Analise_Sentimento_de_Redes_Sociais/blob/main/Analise_Sentimento_Twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U google-generativeai

In [ ]:
pip install tweepy -q -U

In [ ]:
import tweepy
import pandas as pd
from textblob import TextBlob
import datetime
import matplotlib.pyplot as plt
from google.colab import userdata

# Credenciais do Twitter (substitua com suas credenciais)
consumer_key = userdata.get("twapikey")
consumer_secret = userdata.get("twapisecret")
access_token = userdata.get("twacesstok")
access_token_secret = userdata.get("twacesssecrettok")

# Autenticar na API do Twitter
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

def analyze_sentiment(text):
  """Analisa o sentimento de um texto usando TextBlob."""
  blob = TextBlob(text)
  return blob.sentiment.polarity

def get_tweets(hashtag, hours=24):
  """Coleta tweets com a hashtag especificada nas últimas 'hours' horas."""
  now = datetime.datetime.utcnow()
  since = now - datetime.timedelta(hours=hours)
  tweets = tweepy.Cursor(api.search_tweets, q=hashtag, lang='pt', tweet_mode='extended', since=since.strftime('%Y-%m-%d')).items(100)
  return tweets

def main():
  """Função principal para coletar, analisar e visualizar sentimentos de tweets."""
  hashtag = input("Digite a hashtag que deseja analisar: ")

  tweets = get_tweets(hashtag)

  tweets_data = [[tweet.created_at, tweet.full_text] for tweet in tweets]
  df_tweets = pd.DataFrame(tweets_data, columns=['Data', 'Tweet'])

  df_tweets['Sentimento'] = df_tweets['Tweet'].apply(analyze_sentiment)

  # Calcular a média de sentimento
  media_sentimento = df_tweets['Sentimento'].mean()
  print(f"Média de Sentimento para #{hashtag}: {media_sentimento:.2f}")

  # Visualização
  plt.figure(figsize=(10, 5))
  plt.plot(df_tweets['Data'], df_tweets['Sentimento'], label='Sentimento')
  plt.title(f'Análise de Sentimento para #{hashtag} nas últimas 24 horas')
  plt.xlabel('Data')
  plt.ylabel('Score de Sentimento')
  plt.legend()
  plt.show()

if __name__ == "__main__":
  main()

In [ ]:
from typing import Any
import tweepy
import pandas as pd
from textblob import TextBlob
import datetime
import matplotlib.pyplot as plt
import google.generativeai as genai

from google.cloud import aiplatform

from google.colab import userdata

GOOGLE_API_KEY = userdata.get('GeminiAI')

#aiplatform.init(project=config.PROJECT_ID, location=config.REGION, api_key=config.GOOGLE_API_KEY)
aiplatform.init()

# Credenciais do Twitter (substitua com suas credenciais)
bearer_token = userdata.get('twbearer')
consumer_key = userdata.get("twapikey")
consumer_secret = userdata.get("twapisecret")
access_token = userdata.get("twacesstok")
access_token_secret = userdata.get("twacesssecrettok")



# Criar cliente da API v2
client = tweepy.Client(bearer_token=bearer_token)
# api = tweepy.API(auth)

''' # Caso queira usar o Blob para analisar o sentimento
def analyze_sentiment(text):
  """Analisa o sentimento de um texto usando TextBlob."""
  blob = TextBlob(text)
  return blob.sentiment.polarity
'''
def analyze_sentiment(text):
    """Analisa o sentimento de um texto usando o Gemini."""
    response = aiplatform.gapic.ModelServiceClient().predict(
        endpoint=config.GEMINI_ENDPOINT,
        instances=[{"content": text}],
        parameters={"temperature": 0.0},
    )

    sentiment = response.predictions[0]['sentiment']
    return sentiment['score']

def get_tweets(hashtag, hours=24, max_results=10):
  """Coleta tweets com a hashtag especificada nas últimas 'hours' horas."""
  now = datetime.datetime.utcnow()
  since = now - datetime.timedelta(hours=hours)
  tweets = []

  # Parâmetros da busca
  query = f'{hashtag} lang:pt -is:retweet'
  start_time = since.isoformat()
  end_time = now.isoformat()


  for tweet in tweepy.Paginator(client.search_recent_tweets, query=query,
                                start_time=start_time, end_time=end_time,
                                tweet_fields=['created_at', 'text'], max_results=100).flatten(limit=1000):
    tweets.append([tweet.created_at, tweet.text])

  return tweets

'''
  tweets = client.search_recent_tweets(query=query,
                                    start_time=start_time,
                                    end_time=end_time,
                                    tweet_fields=['created_at', 'text'],
                                    max_results=max_results).data
'''

def main():
  """Função principal para coletar, analisar e visualizar sentimentos de tweets."""
  hashtag = input("Digite a hashtag que deseja analisar: ")

  tweets = get_tweets(hashtag, max_results=10)

  if tweets:
    df_tweets = pd.DataFrame(tweets, columns=['Data', 'Tweet'])
    df_tweets['Sentimento'] = df_tweets['Tweet'].apply(analyze_sentiment)

    # Calcular a média de sentimento
    media_sentimento = df_tweets['Sentimento'].mean()
    print(f"Média de Sentimento para #{hashtag}: {media_sentimento:.2f}")

    # Visualização
    plt.figure(figsize=(10, 5))
    plt.plot(df_tweets['Data'], df_tweets['Sentimento'], label='Sentimento')
    plt.title(f'Análise de Sentimento para #{hashtag} nas últimas 24 horas')
    plt.xlabel('Data')
    plt.ylabel('Score de Sentimento')
    plt.legend()
    plt.show()
  else:
    print(f"Nenhum tweet encontrado para #{hashtag} nas últimas 24 horas.")

if __name__ == "__main__":
  main()

In [ ]:
import tweepy
from google.colab import userdata

CONSUMER_KEY = userdata.get('twapikey')
CONSUMER_SECRET = userdata.get('twapisecret')
ACCESS_TOKEN = userdata.get('twacesstok')
ACCESS_TOKEN_SECRET = userdata.get('twacesssecrettok')

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

status = "Testing!"
api.update_status(status=status)